# Minitarea 2

Nombre: Ignacio Díaz Lara


---------------------------
## Language Models (3 pts)
Estas preguntas son teóricas y deben ser resueltas con desarrollo, pero sin código. Por favor usen $\LaTeX$ para las fórmulas. El material del curso y los videos deberian ser suficientes para que puedan responder.


### Pregunta 1 (1 pt)
Asuma que tiene un modelo de lenguaje de trigramas (simple) entrenado sobre un corpus C. Muestre cómo el modelo le asigna una probabilidad a la oración `el perro le ladra al gato` usando los parámetros estimados a partir del corpus  $q(w_i | w_{i-2}, w_{i-1})$ .

**Respuesta:**

Distribuimos la probabilidad como $p(x_{1}...x_{n}) = \prod_{i=1}^{n}q(x_{i}|x_{i-2},x_{i-1})$ donde cada trigrama se define como $q(w_{i}|w_{i-2},w_{i-1}) = \frac{Count(w_{i-2},w_{i-1},w_{i})}{Count(w_{i-2},w_{i-1})}$


En particular, la probabilidad asignada para la oración "el perro le ladra al gato":

$p($el perro le ladra al gato STOP$) =$

$q(el|*,*) \cdot q(perro|*,el) \cdot q(le|el,perro) \cdot q(ladra|perro,le) \cdot q(al|le,ladra) \cdot q(gato|ladra,al) \cdot q(STOP|al,gato) $

=$\frac{Count(*,*,el)}{Count(*,*)} \cdot \frac{Count(*,el,perro)}{Count(*,el)} \cdot \frac{Count(el,perro,le)}{Count(el,perro)} \cdot \frac{Count(perro,le,ladra)}{Count(perro,le)} \cdot \frac{Count(le,ladra,al)}{Count(le,ladra)} \cdot \frac{Count(ladra,al,gato)}{Count(ladra,al)} \cdot \frac{Count(al,gato,STOP)}{Count(al,gato)} $


Todos esos Count se hacen sobre el corpus C.

### Pregunta 2 (1 pt)
Muestre cómo se calcularía  $q(w_{i} | w_{i-2}, w_{i-1})$  usando un modelo que interpola un modelo de lenguajes de trigramas, bigramas, y unigrama.


**Respuesta:**

La probabilidad se distribuye igual que en la pregunta anterior, pero en este caso definimos los trigramas como 

$q(w_{i}|w_{i-2},w_{i-1}) = \lambda_{1} \cdot q_{ML}(w_{i}|w_{i-2},w_{i-1}) + \lambda_{2} \cdot q_{ML}(w_{i}|w_{i-1}) + \lambda_{3} \cdot q_{ML}(w_{i})$

Con $\lambda_{1}+\lambda_{2}+\lambda_{3} = 1$, y $\lambda_{i}\geq0$ para todo $i$

Con las siguientes definiciones de los trigramas, bigramas y unigramas de máxima verosimilitud:

$q_{ML}=(w_{i}|w_{i-2},w_{i-1}) = \frac{Count(w_{i-2},w_{i-1},w_{i})}{Count(w_{i-2},w_{i-1})}$

$q_{ML}=(w_{i}|w_{i-1}) = \frac{Count(w_{i-1},w_{i})}{Count(w_{i-1})}$

$q_{ML}=(w_{i}) = \frac{Count(w_{i})}{Count()}$

Notar que los Count buscan trigramas, bigramas o unigramas en el corpus C. Pero $Count()=N$ con $N$ el tamaño del corpus.

Con respecto a $\lambda_{i}$, hay distintas formas de estimar sus valores. Por ejemplo, suponiendo que se tienen más datos separados de los datos de entrenamiento y del corpus, llámese 'development data' y definimos $c'(u,v,w)$ como el número de veces que aparece el trigrama $(u,v,w)$. Teniendo esto, la función logarítmica de verosimilitud $L$ sería:

$L(\lambda_{1},\lambda_{2},\lambda_{3})=\sum_{u,v,w}Count'(u,v,w)$ $log$ $q(w|u,v)=\sum_{u,v,w}Count'(u,v,w)$ $log(\lambda_{1} \cdot q_{ML}(w|u,v) + \lambda_{2} \cdot q_{ML}(w|v) + \lambda_{3} q_{ML}(w))$

Luego para optimizar los valores de $\lambda_{1},\lambda_{2},\lambda_{3}$ maximizamos $L$:

$arg$ $max_{\lambda_{1},\lambda_{2},\lambda_{3}}$ $L(\lambda_{1},\lambda_{2},\lambda_{3})$ 

s.a. $\lambda_{1}+\lambda_{2}+\lambda_{3} = 1$, y $\lambda_{i}\geq0$ para $i \in \{1,2,3\}$

### Pregunta 3 (1 pt)
¿Qué ventajas tiene el modelo interpolado sobre el modelo de trigramas simple?

**Respuesta:**

La principal ventaja es que en el modelo interpolado habrá muchos menos indefiniciones que en el de trigrama simple. Esto debido a que en el modelo simple, si un trigrama no aparece nunca en los datos de entrenamiento entonces el valor de $Count(w_{i-2},w_{i-1},w_{i})$ será 0, lo que no aporta nada, pero peor aún, cuando $Count(w_{i-2},w_{i-1})$ sea cero, es decir que el bigrama que representa no aparezca en los datos de entrenamiento, entonces habrá muchos expresiones que se indefinirán al ser dividadas por cero. El modelo interpolado soluciona esto con la inclusión de los $\lambda_{i}$, y para los casos en que lo representado por la expresión no aparezca en los datos de entrenamiento, su $\lambda_{i}$ correspondiente será 0, evitando la indefinición como en el modelo del trigrama simple.

-----------------------
## Naive Bayes (3 pts)
En esta parte de la minitarea deberan programar Naive Bayes Multinomial usando Laplace Smothing. Las referencias las pueden encontrar en el material del curso y los videos del profesor.

A continuacion se presentan un conjunto de documentos de training divididos en 2 categorias distintas. Ustedes deberan clasificar los documentos del test set usando Naive Bayes con Laplace Smothing.

Por favor, documenten su codigo. Escriban que hacen las funciones, que reciben, que entregan, etc. Si en algun lugar de su codigo hacen algo que creen que debe ser explicado, pongan comentarios, son bienvenidos. 


**Underflow prevention:** En vez de hacer muchas multiplicacions de `float`s, reemplacenlas por sumas de logaritmos para prevenir errores de precision. Revisen la diapo 69 de las slides. 

NOTA: Sobre las `namedtuple`s. Es el tipo de los documentos. Son objetos inmutable que tienen dos atributos: `words` donde estan las palabras del documento y `class_` donde se guarda la clase de ese documento. Estos objetos son inmutables, lo que quiere decir que si quieren modificar un documento y cambiarle la clase, tienen que crear otro documento. Otra cosa es que son tuplas como cualquier otra, es decir se pueden acceder usando indices como `doc[0]` o `doc[1]`. Son libres de implementar su solucion como quieran, si no les gusta este tipo de dato usen otro.



In [1]:
from pprint import pprint
from collections import namedtuple
document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

train_set = (
    document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
    document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
    document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
    document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
    document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
    document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
    document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
    document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1)
)
print("Train documents:")
pprint(train_set)


test_set = (document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)
print("\nTest documents:")
pprint(test_set)

Train documents:
(document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
 document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
 document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
 document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
 document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
 document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
 document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
 document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1))

Test documents:
(document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)


In [0]:
# Tu respuesta